In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%env OPENAI_API_KEY=<Enter you key here>

env: OPENAI_API_KEY=<Enter you key here>


In [3]:
from PIL import Image
from IPython.core.display import HTML
from functools import partial

from engine.utils import ProgramGenerator, ProgramInterpreter
from prompts.nlvr import create_prompt

In [4]:
interpreter = ProgramInterpreter(dataset='nlvr')

Registering VQA step
Registering EVAL step
Registering RESULT step


In [5]:
prompter = partial(create_prompt,method='all')
generator = ProgramGenerator(prompter=prompter)

In [6]:
left_image = Image.open('../assets/camel1.png')
left_image.thumbnail((640,640),Image.Resampling.LANCZOS)
right_image = Image.open('../assets/camel2.png')
right_image.thumbnail((640,640),Image.Resampling.LANCZOS)
init_state = dict(
    LEFT=left_image.convert('RGB'),
    RIGHT=right_image.convert('RGB'),
)

In [7]:
statement = "There is a woman in black in exactly one of the images and two camels in total."
# statement = "There are more camels than people."
# statement = "The camel is sitting in both images."
# statement = "There are no trees in the images."
prog,_ = generator.generate(dict(statement=statement))
print(prog)

ANSWER0=VQA(image=LEFT,question='Is there a woman in black in the image?')
ANSWER1=VQA(image=RIGHT,question='Is there a woman in black in the image?')
ANSWER2=VQA(image=LEFT,question='How many camels are in the image?')
ANSWER3=VQA(image=RIGHT,question='How many camels are in the image?')
ANSWER4=EVAL(expr='{ANSWER0} xor {ANSWER1}')
ANSWER5=EVAL(expr='{ANSWER2} + {ANSWER3} == 2')
ANSWER6=EVAL(expr='{ANSWER4} and {ANSWER5}')
FINAL_ANSWER=RESULT(var=ANSWER6)


In [8]:
result, prog_state, html_str = interpreter.execute(prog,init_state,inspect=True)

VQA


/Users/sanideshmukh/miniforge3/envs/visprog/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


VQA
VQA
VQA
EVAL
EVAL


TypeError: can only concatenate str (not "int") to str

In [ ]:
result

In [ ]:
HTML(html_str)